In [1]:
# Modify the file 00_setup.py to define input/output file paths on your system
# The information in 00_setup.py will be used across notebooks
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./00_setup.py").load_module()

# GNN - Data for Unsupervised Model

Create Stellargraph objects for unsupervised GNN runs.  The goal is to get embeddings for use in e.g. XGBoost models.

Here I use raw training features, with no label feature information

The unsupervised model has both business and NAICS nodes, and uses NAICS nodes as the head type.

*This script takes about 15 minutes on my MacBook Air*

In [2]:
import pandas as pd
import numpy as np

In [3]:
from pathlib import Path
import importlib
import pickle, re

In [4]:
from sba_gnn.sba_gnn import sg_gnn

In [5]:
import stellargraph as sg
from stellargraph import StellarGraph

2024-01-29 08:19:38.098559: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-29 08:19:38.098585: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-29 08:19:38.098591: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-29 08:19:38.098628: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-29 08:19:38.098645: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Import Data

In [6]:
## NN scaled input data
comb_df = pd.read_parquet(Path(setup.temp_path).joinpath('10_DATA_combined_scaled_all.parquet'))

In [7]:
with open(Path(setup.temp_path).joinpath('10_DATA_features.pkl'), 'rb') as fin:
    imputed_features = pickle.load(fin)

In [8]:
num_feat =  [c for c in imputed_features if c != 'NAICS']
print(num_feat)

['NoEmp', 'CreateJob', 'LowDoc', 'DisbursementGross', 'new_business', 'urban_flag', 'franchise_flag', 'missingindicator_LowDoc', 'missingindicator_new_business', 'missingindicator_urban_flag']


## Create Edge Map

I will have edges between a business and its NAICS, as well as edges to same-sector NAICS nodes.  This section creates general edge data, which may be filtered to create graphs

##### Create NAICS map to same-sector NAICS

In [17]:
sector_map = comb_df[['NAICS_orig', 'NAICS_sector']].drop_duplicates()

In [18]:
naics_all_sectors = sector_map.merge(sector_map.rename(columns={'NAICS_orig':'NAICS_sim'}),
                                     how='left', on ='NAICS_sector')
naics_all_sectors = naics_all_sectors[naics_all_sectors['NAICS_orig'] != naics_all_sectors['NAICS_sim']]

In [19]:
naics_all_sectors.sample(5)

,NAICS_orig,NAICS_sector,NAICS_sim
266634,325182,31-33,322231
192017,311511,31-33,337910
66624,337125,31-33,311422
138325,315292,31-33,325612
205005,332997,31-33,334611


##### Functions for creating index for NAICS

##### Get edges from businesses to their exact NAICS

In [21]:
# Edges from businesses to their NAICS code
naics_info_df = comb_df[['LoanNr_ChkDgt', 'NAICS_orig', 'dset']].copy() \
    .rename(columns={'LoanNr_ChkDgt':'source'}) 
naics_info_df['target'] = sg_gnn.get_naics_index(naics_info_df['NAICS_orig'])

edges_business_naics = naics_info_df[['source', 'target', 'dset']].copy()
edges_business_naics['type'] = 'loan_naics'

##### Get edges from buisinesses to related NAICS codes

In [22]:
naics_info_df.head(3)

,source,NAICS_orig,dset,target
index,,,,
0,1000014003,451120,train,n_451120
2,1000034009,621210,train,n_621210
7,1000094005,811118,train,n_811118


In [23]:
edges_businesses_naics_sim = naics_info_df.drop(columns='target', errors='ignore') \
    .merge(naics_all_sectors, how='inner', on='NAICS_orig')
edges_businesses_naics_sim['target'] = sg_gnn.get_naics_index(edges_businesses_naics_sim['NAICS_sim'])
edges_businesses_naics_sim = edges_businesses_naics_sim[['source', 'target', 'dset']]
edges_businesses_naics_sim['type'] = 'loan_sector_naics'
print(f'Sector NAICS edges: {edges_businesses_naics_sim.shape}')

Sector NAICS edges: (67912366, 4)


##### Sample these - See 00_setup.py
The data is large so sampling could be considered here.  This is controlled in 00_setup.py.

In [26]:
if setup.gnn_unsup_sample:
    edges_businesses_naics_sim = edges_businesses_naics_sim.groupby('source') \
        .sample(setup.gnn_unsup_sample_n, replace=True, random_state=2342) \
        .drop_duplicates()
    print(f'Sector NAICS edges post sample: {edges_businesses_naics_sim.shape}')

Sector NAICS edges post sample: (22483771, 4)


In [31]:
edges_all = pd.concat([edges_business_naics, edges_businesses_naics_sim], axis=0) \
    .reset_index(drop=True)

In [32]:
edges_all['type'].value_counts()

type
loan_sector_naics    22483771
loan_naics             688081
Name: count, dtype: int64

In [33]:
print(edges_all[['source', 'target']].drop_duplicates().shape)
print(edges_all.shape)

(23171852, 2)
(23171852, 4)


In [34]:
edges_all.to_parquet(Path(setup.temp_path).joinpath('40_DATA_graph_edges.parquet'))

## Get Node Features
Use the scaled NN data.  For NAICS, just set an indicator for now.  

In [9]:
features_business = comb_df[['LoanNr_ChkDgt', 'dset', 'NAICS_orig'] + num_feat] \
    .set_index('LoanNr_ChkDgt')
features_business.to_parquet(Path(setup.temp_path).joinpath('40_DATA_graph_node_features_loans.parquet'))

In [10]:
# NAICS features - Just use indicator.  Include original code for mapping
features_naics = comb_df[['NAICS_orig']].drop_duplicates()
features_naics['feat'] = 1
features_naics['source'] = sg_gnn.get_naics_index(features_naics['NAICS_orig'])
features_naics.set_index('source', inplace=True) 
print(features_naics.shape)
features_naics.to_parquet(Path(setup.temp_path).joinpath('40_DATA_graph_node_features_naics.parquet'))

(1311, 2)


## Stellargraph - train and validation
Graph without test nodes (make sure set-aside NAICS not included)

In [57]:
# Get only relevant edges and node data
edges_train_val, features_business_train_val, features_naics_train_val = \
    sg_gnn.limit_data(edges_all, features_business, features_naics.drop(columns='NAICS_orig'),
                     ['train', 'val'])

In [58]:
print(f'Train+val graph data')
print(f'business features, start rows {features_business.shape[0]}, end {features_business_train_val.shape[0]}')
print(f'naics features, start rows {features_naics.shape[0]}, end {features_naics_train_val.shape[0]}')
print(f'edges, start rows {edges_all.shape[0]}, end {edges_train_val.shape[0]}')

Train+val graph data
business features, start rows 688081, end 551635
naics features, start rows 1311, end 1240
edges, start rows 23171852, end 17688739


In [68]:
sba_graph_train_val = StellarGraph({'LoanNr_ChkDgt':features_business_train_val.drop(columns='NAICS_orig'),
                                    'NAICS': features_naics_train_val},
                                   edges_train_val, 
                                   source_column="source", target_column="target",
                                   edge_type_column="type")

In [69]:
print(sba_graph_train_val.info())

StellarGraph: Undirected multigraph
 Nodes: 552875, Edges: 17688739

 Node types:
  LoanNr_ChkDgt: [551635]
    Features: float32 vector, length 10
    Edge types: LoanNr_ChkDgt-loan_naics->NAICS, LoanNr_ChkDgt-loan_sector_naics->NAICS
  NAICS: [1240]
    Features: float32 vector, length 1
    Edge types: NAICS-loan_naics->LoanNr_ChkDgt, NAICS-loan_sector_naics->LoanNr_ChkDgt

 Edge types:
    LoanNr_ChkDgt-loan_sector_naics->NAICS: [17137104]
        Weights: all 1 (default)
        Features: none
    LoanNr_ChkDgt-loan_naics->NAICS: [551635]
        Weights: all 1 (default)
        Features: none


In [70]:
# Save stellargraph object
with open(Path(setup.temp_path).joinpath('40_DATA_stellargraph_train_val.pkl'), 'wb') as fout:
      pickle.dump(sba_graph_train_val, fout)

In [77]:
features_naics.columns

Index(['feat'], dtype='object')

## Stellargraph - all nodes

In [78]:
sba_graph_all = StellarGraph({'LoanNr_ChkDgt':features_business.drop(columns=['dset', 'NAICS_orig']) ,
                              'NAICS': features_naics.drop(columns='NAICS_orig')},
                             edges_all.drop(columns=['dset']).drop_duplicates(),
                             source_column="source", target_column="target",
                             edge_type_column="type")

In [79]:
print(sba_graph_all.info())

StellarGraph: Undirected multigraph
 Nodes: 689392, Edges: 23171852

 Node types:
  LoanNr_ChkDgt: [688081]
    Features: float32 vector, length 10
    Edge types: LoanNr_ChkDgt-loan_naics->NAICS, LoanNr_ChkDgt-loan_sector_naics->NAICS
  NAICS: [1311]
    Features: float32 vector, length 1
    Edge types: NAICS-loan_naics->LoanNr_ChkDgt, NAICS-loan_sector_naics->LoanNr_ChkDgt

 Edge types:
    LoanNr_ChkDgt-loan_sector_naics->NAICS: [22483771]
        Weights: all 1 (default)
        Features: none
    LoanNr_ChkDgt-loan_naics->NAICS: [688081]
        Weights: all 1 (default)
        Features: none


In [80]:
# Save stellargraph object
with open(Path(setup.temp_path).joinpath('40_DATA_stellargraph_all.pkl'), 'wb') as fout:
      pickle.dump(sba_graph_all, fout)